# Query Construction

> Query construction is the process of transforming a natural language query into the query language of the database or data source you are interacting with.

### Text-Metadata

In [1]:
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [10]:
import os
import dotenv

dotenv.load_dotenv()

embeddings = OllamaEmbeddings(
    base_url="http://127.0.0.1:11434", model="deepseek-r1:8b"
)

os_key = os.getenv("OPENSEARCH_INITIAL_ADMIN_PASSWORD")

db = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    opensearch_url="https://localhost:9200",
    http_auth=("admin", os_key),
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    engine="faiss",
)

In [11]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_ollama import ChatOllama

fields = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
description = "Brief summary of a movie"

llm = ChatOllama(model="deepseek-r1:8b",temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm, db, description, fields,
)

!!!!  IMPORTANT: Need capable models

In [12]:
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(id='2623a555-c1b7-44fd-86f1-142c48a44fd8', metadata={'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='61fb1a3d-803c-43cf-b886-540a9dd5b2ee', metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

### Text-SQL

> * Database Description
> * Few Shot examples

In [20]:
from langchain.pydantic_v1 import BaseModel

class AnswerAndQuery(BaseModel):
    '''An answer to the user's question along with ju
        answer.'''
    answer: str
    '''The answer to the user's question'''
    query: str
    '''SQL Query'''

In [ ]:
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain_ollama import ChatOllama


# replace this with the connection details of your db
db = SQLDatabase.from_uri("sqlite:///../data/sample.db")
llm = ChatOllama(model="qwen:0.5b", temperature=0)

# convert question to sql query
write_query = create_sql_query_chain(llm, db)

# Execute SQL query
execute_query = QuerySQLDatabaseTool(db=db)

# combined
chain = write_query | execute_query

# invoke the chain
chain.invoke({"question":"How many employees are there?"})

'Error: (sqlite3.OperationalError) near "To": syntax error\n[SQL: To determine the number of employees, you would need to join the `Users` table with other tables in your database. Once you have joined all the tables, you can use the COUNT function to count the number of rows in the joined table.\nAnswer: There are 3 rows from the `Users` table.]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [20]:
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import ChatPromptTemplate, AIMessagePromptTemplate
from langchain_ollama import ChatOllama

my_prompt = """
Given an input question create a SQL query according to question. Only return SQL query nothing else \n
Question: {input}
"""

my_prompt = ChatPromptTemplate.from_template(my_prompt)


# replace this with the connection details of your db
db = SQLDatabase.from_uri("sqlite:///../data/sample.db")
llm = ChatOllama(model="qwen:0.5b", temperature=0)

# convert question to sql query
write_query = my_prompt | llm

# Execute SQL query
execute_query = QuerySQLDatabaseTool(db=db)

# Get content
def get_content(msg:AIMessagePromptTemplate):
    return msg.content

# combined
chain = write_query | get_content |execute_query

# invoke the chain
chain.invoke({"input":"Can you list all the users?"})

"[(1, 'acf', '123'), (2, 'ale', '456'), (3, 'abc', '789'), (4, 'zyx', '987')]"